# Homework 2

Do the programing part of Homework 2 in this notebook. Predefined are function *stubs*. That is, the name of the function and a basic body is predefined. You need to modify the code to fulfil the requirements of the homework.

In [ ]:
# import numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt

# If you're using Firefox, remove one or both # on the commands below.
# note these `%`-commands are not actually Python commands. They are Jupyter-notebook-specific commands.
#%matplotlib notebook
#%matplotlib notebook

## Network SIR Model

### 1-2 One population

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt


def SIR_rhs(x, t, param): # Note the order of arguments for odeint
    S = x[0]
    I = x[1]
    R = x[2]

    dxdt = np.array([
        -param['beta'] * S * I,
        param['beta'] * S * I - param['gamma'] * I,
        param['gamma'] * I
    ])
    return dxdt




# parameters
param = {
    'N': 100,         # total population size
    'beta': 2.5 / 100,  # infection rate, 1/beta = typical time between contacts
    'gamma': 1.0       # recovery rate, 1/gamma = typical time until recovery
}

# final time
tf = 10

# initial condition
x0 = param['N'] * np.array([0.950, 0.050, 0.000])

R0 = param['beta'] * x0[0] / param['gamma']

# simulate
t = np.linspace(0, tf, 100)  # Create a time vector for odeint
x = odeint(SIR_rhs, x0, t, args=(param,)) # Solve the ODEs

S = x[:, 0]
I = x[:, 1]
R = x[:, 2]

# plot
clrs = np.array([[0, 0, 240], [220, 20, 60], [60, 60, 60]]) / 256

fig, axs = plt.subplots(3, 1)

# time courses
axs[0].plot(t, S, color=clrs[0], linewidth=3)
axs[0].plot(t, I, color=clrs[1], linewidth=3)
axs[0].plot(t, R, color=clrs[2], linewidth=3)
axs[0].legend(['S', 'I', 'R'])
axs[0].set_xlabel('time')
axs[0].set_ylabel('compartments')
axs[0].set_title(f'$R_0$ = {R0}')


# time courses - using stackplot
axs[1].stackplot(t, S, I, R, colors=clrs, labels=['S', 'I', 'R'])
axs[1].legend(loc='upper right') # Adjust legend location as needed
axs[1].set_xlabel('time')
axs[1].set_ylabel('compartments')
axs[1].set_title(f'$R_0$ = {R0}')

# ternary plot
axs[2].plot(0.5 * (2 * R + I) / param['N'], np.sqrt(3) / 2 * I / param['N'], '-k', linewidth=2)
axs[2].plot([0, 0.5, 1, 0], [0, np.sqrt(3) / 2, 0, 0], '-k', linewidth=1)
axs[2].set_xticks([])
axs[2].set_yticks([])
axs[2].set_xlim([-0.1, 1.1])
axs[2].set_ylim([-0.1, np.sqrt(3) / 2 + 0.1])
axs[2].set_aspect('equal')

axs[2].text(0.5, np.sqrt(3) / 2, 'I=N', verticalalignment='bottom', horizontalalignment='center')
axs[2].text(1, 0, 'R=N', verticalalignment='top', horizontalalignment='left')
axs[2].text(0, 0, 'S=N', verticalalignment='top', horizontalalignment='right')

axs[2].spines['top'].set_visible(False) #added to remove the top border
axs[2].spines['right'].set_visible(False) #added to remove the right border
axs[2].spines['bottom'].set_visible(False) #added to remove the bottom border
axs[2].spines['left'].set_visible(False) #added to remove the left border

plt.tight_layout() # Adjust subplot params for a tight layout
plt.show()


### 3-7 Multiple populations

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

#### Define the RHS of the system

In [ ]:
def multipopulation_sir_rhs(x, t, param):
    N = param['N']
    beta = param['beta']
    gamma = param['gamma']
    TS = param['TS']
    TI = param['TI']
    TR = param['TR']
    num_populations = param['num_populations']

    dxdt = np.zeros_like(x)

    for i in range(num_populations):
        S = x[i * 3]
        I = x[i * 3 + 1]
        R = x[i * 3 + 2]

        dSdt = -beta[i] * S * I  # Use beta[i] and N[i]
        dIdt = beta[i] * S * I - gamma[i] * I  # Use gamma[i] and N[i]
        dRdt = gamma[i] * I  # Use gamma[i]

        # Travel component
        for j in range(num_populations):
            if i != j:  # Don't travel to the same population
                dSdt += TS[j, i] * x[j * 3] - TS[i, j] * S  # Use N[i] and N[j]
                dIdt += TI[j, i] * x[j * 3 + 1] - TI[i, j] * I  # Use N[i] and N[j]
                dRdt += TR[j, i] * x[j * 3 + 2] - TR[i, j] * R  # Use N[i] and N[j]

        dxdt[i * 3] = dSdt
        dxdt[i * 3 + 1] = dIdt
        dxdt[i * 3 + 2] = dRdt

    return dxdt

#### Parameters, initial values, travelling matrix

In [ ]:

# Parameters
num_populations = 4  # number of different populations

# Define N, beta, and gamma as arrays
N = np.array([100, 100, 100, 100])  # Population size for each subpopulation
beta = np.array([2, 0.5, 2, 1.5])/N  # Infection rate for each subpopulation
gamma = np.array([1.0, 1.0, 1.0, 1.0])  # Recovery rate for each subpopulation
tf = 20  # Final time

# Initial conditions (all populations start with the same infection rate)
x0 = np.zeros([num_populations, 3])
x0[:, 0] = N
x0[-1, :] = N[-1] * np.array([0.95, 0.05, 0])

R0 = beta * x0[:, 0] / gamma #per population R0

x0 = x0.ravel()

# Set up the travel matrices (same as before)
nu = 0.3  # travel rate

Tall = np.ones((num_populations, num_populations))
Tnone = np.zeros((num_populations, num_populations))
Tchain = np.diag(np.ones(num_populations - 1), 1) + np.diag(np.ones(num_populations - 1), -1)
Tonly1 = np.block([[0, np.ones(num_populations - 1)], [np.ones((num_populations - 1, 1)), np.zeros((num_populations - 1, num_populations - 1))]])
Tcir = np.diag(np.ones(num_populations - 1), 1)
Tcir[num_populations - 1, 0] = 1

TS = nu * Tall
TI = nu * Tnone
TR = nu * Tnone


#### See the current travelling scheme

In [ ]:
import networkx as nx


for compartment, matrix, nu_val in [("S", TS, nu), ("I", TI, nu), ("R", TR, nu)]:
    if np.any(matrix):  # Check if the matrix has any non-zero elements
        G = nx.DiGraph();  # Create a directed graph for each compartment

        for i in range(num_populations):
            G.add_node(f"P{i+1}")

        for i in range(num_populations):
            for j in range(num_populations):
                if i != j and matrix[i, j] > 0:  # Add edge only if travel exists
                    G.add_edge(f"P{i+1}", f"P{j+1}", label=f"{compartment}")

        if G.number_of_edges() > 0:  # Only plot if there are edges
            pos = nx.circular_layout(G)
            nx.draw(G, pos, with_labels=True, node_size=2000, node_color="skyblue", font_size=16, arrowsize=20); # Straight arrows by default

            edge_labels = nx.get_edge_attributes(G, 'label')

            label_pos = {}

            for source, target in G.edges():
                count = 0
                for key in G[source][target]:
                    label = edge_labels.get((source, target, key))
                    if label:
                        x_source, y_source = pos[source]
                        x_target, y_target = pos[target]

                        x_mid = (x_source + x_target) / 2
                        y_mid = (y_source + y_target) / 2

                        angle = np.arctan2(y_target - y_source, x_target - x_source)

                        offset_magnitude = 0.15  # Adjust for offset distance
                        offset_x = offset_magnitude * np.cos(angle + np.pi / 2) * (G.number_of_edges(source, target) - 1)
                        offset_y = offset_magnitude * np.sin(angle + np.pi / 2) * (G.number_of_edges(source, target) - 1)

                        label_pos[(source, target, key)] = (x_mid + offset_x, y_mid + offset_y)
                        count += 1

            for edge, label in edge_labels.items():
                nx.draw_networkx_edge_labels(G, pos, {edge: label}, font_size=14)

            plt.title(f"Travel Between Populations ({compartment}, $\\nu=${nu_val})");
            plt.show();

plt.tight_layout();
plt.show();

#### Solve the system and plot solution

In [ ]:
param = {'N': N, 'beta': beta, 'gamma': gamma, 'TS': TS, 'TI': TI, 'TR': TR, 'num_populations': num_populations}

# Simulate (same as before)
t = np.linspace(0, tf, 100)
x = odeint(multipopulation_sir_rhs, x0, t, args=(param,))

# Reshape the solution for easier plotting (same as before)
S = x[:, 0::3].reshape(len(t), num_populations)  # Every 3rd element starting at 0 is S
I = x[:, 1::3].reshape(len(t), num_populations)  # Every 3rd element starting at 1 is I
R = x[:, 2::3].reshape(len(t), num_populations)  # Every 3rd element starting at 2 is R

# Plotting (same as before)
clrs = np.array([[0, 0, 240], [220, 20, 60], [60, 60, 60]]) / 256

# Plotting
fig, axs = plt.subplots(3, 1, figsize=(10, 12))  # Add space for the pie chart

# Time series plots (as before)
for i in range(num_populations):
    axs[0].plot(t, S[:, i], color=clrs[0], linewidth=2, label=f'S{i+1}')
    axs[0].plot(t, I[:, i], color=clrs[1], linewidth=2, label=f'I{i+1}')
    axs[0].plot(t, R[:, i], color=clrs[2], linewidth=2, label=f'R{i+1}')
axs[0].set_xlabel('time')
axs[0].set_ylabel('compartments')
# axs[0].legend()
# Format R0 values for the title
r0_strings = [f"{r:.1f}" for r in R0]  # Format each R0 to one decimal place
r0_title = ", ".join(r0_strings)  # Join the strings with commas

axs[0].set_title(f'Initial R0: {r0_title}')  # Use the formatted string in the title


colors = [clrs[0]] * num_populations + [clrs[1]] * num_populations + [clrs[2]] * num_populations

axs[1].stackplot(t, S.T, I.T, R.T, colors=colors, labels=['S', 'I', 'R'])  # Transpose S, I, R
axs[1].set_xlabel('time')
axs[1].set_ylabel('compartments')
# axs[1].legend()
axs[1].set_title(f'Initial R0: {r0_title}')  # Use the formatted string in the title


# Pie chart of final configuration (with per-population labels)
total_population = N.sum()
final_S = S[-1, :]
final_I = I[-1, :]
final_R = R[-1, :]

sizes = np.concatenate([final_S, final_I, final_R])
labels = []
for i in range(num_populations):
    labels.append(f'S{i+1}')
for i in range(num_populations):
    labels.append(f'I{i+1}')
for i in range(num_populations):
    labels.append(f'R{i+1}')

colors = [clrs[0]] * num_populations + [clrs[1]] * num_populations + [clrs[2]] * num_populations

# Filter out small values and corresponding labels/colors
threshold = 0.01 * total_population  # 1% of total population as threshold  (adjust as needed)
filtered_sizes = []
filtered_labels = []
filtered_colors = []

for i in range(len(sizes)):
    if sizes[i] > threshold:
        filtered_sizes.append(sizes[i])
        filtered_labels.append(labels[i])
        filtered_colors.append(colors[i])

axs[2].pie(filtered_sizes, labels=filtered_labels, colors=filtered_colors, autopct='%1.1f%%', startangle=90)
axs[2].axis('equal')
axs[2].set_title('Final Compartment Distribution (Per Population)')

plt.tight_layout()
plt.show()


# Display the final global distribution in a table

print("Final Distribution (% of Global Population)")
print("---------------------------------------")  # Separator line
print("i    S     I     R")

for i in range(num_populations):
    print(f"{i+1:2d} {final_S[i]/N[i]*100:5.1f} {final_I[i]/N[i]*100:5.1f} {final_R[i]/N[i]*100:5.1f}")  # Formatted output

print("---------------------------------------")  # Separator line
print(f"   {final_S.sum()/total_population*100:5.1f} {final_I.sum()/total_population*100:5.1f} {final_R.sum()/total_population*100:5.1f}") # Global totals



## 8-14 Approximating the temperature of a two-dimensional slab

$$u_{xx} + u_{yy} = 0$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm


# We will work with the case: Delta x = Delta y = Delta

xmin=0
xmax=4
ymin=0
ymax=3
Delta = 1


Nx = int((xmax-xmin)/Delta)
Ny = int((ymax-ymin)/Delta)

x=np.linspace(xmin,xmax,Nx)
y=np.linspace(ymin,ymax,Ny)
x = x[1:Nx-1]  # We only use interior points
y = y[1:Ny-1]  # We only use interior points
Nx-=1
Ny-=1
N = Nx*Ny

# The matrix A should have dimension N x N
# The vector b should have dimension N
#
# TODO: Use this space to create your matrix A and vector b






# End creation of A and b

# Solve the linear system
sol = np.linalg.solve(A,b);
sol = sol.reshape(Ny, Nx).T


# Put the edges back in the graph
# u is the matrix with the temperatures going all the way to the edges

Nx+=2  # add both left and right edge points
Ny+=2  # add both bottom and top edge points
u = np.zeros([Nx,Ny])

for i in range(1,Nx-1):
    #u[i,j] = 0
    for j in range(1,Ny-1):
        u[i,j] = sol[i-1,j-1]
    u[i,Ny-1] = 100
for j in range(1,Ny):
    u[Nx-1,j] = 50
u[Nx-1,Ny-1] = 100
# u[0,Ny-1] = 100


# Plot the temperatures as different colours

x=np.linspace(xmin,xmax,Nx)
y=np.linspace(ymin,ymax,Ny)
[mX,mY] = np.meshgrid(x,y)

fig, axs = plt.subplots(1, 1, figsize=(10, 6))  # Add space for the pie chart

nlevels=50
plt.contourf(mX,mY,u.T, nlevels, cmap=cm.coolwarm);
plt.colorbar();
plt.axis('equal');

In [ ]:
# Plot the temperatures as different heights

fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10, 6))

# # Calculate the profit function at all values we are going to plot
# Z = profit(X,Y)
surf=ax.plot_surface(mX,mY,u.T,cmap=cm.coolwarm);
ax.view_init(40, -110);
fig.colorbar(surf, shrink=0.75);

## 15 *(Bonus)* Approximating the temperature of a two-dimensional slab
